In [1078]:
import math
from tabulate import tabulate
from itertools import combinations, permutations


locations = [
    "DC",
    "Krónan Grafarholt",
    "Krónan Mosfellsbær",
    "Krónan Bíldshöfi",
    "Krónan Árbær",
    "Krónan Jafnarsel",
    "Krónan Vallarkór",
    "Krónan Lindir",
    "Krónan Grandi",
    "Krónan Hallveigarstígur",
    "Krónan Borgartún",
    "Krónan Austurver",
    "Krónan Hamraborg",
    "Krónan Garðabær",
    "Krónan Flatahraun",
    "Krónan Hvaleyrarbraut", 
    "Krónan Norðurhella",
    "Krónan Skeifan"
]

# Locations and their coordinates
locations_coords = {
    "DC": (64.12238188422718, -21.80556258041812),
    "Krónan Grafarholt": (64.13060762774467, -21.7616648461748),
    "Krónan Mosfellsbær": (64.16676053874438, -21.69815013762845),
    "Krónan Bíldshöfi": (64.12663767072458, -21.81462238088484),
    "Krónan Árbær": (64.11727231871421, -21.78990314307976),
    "Krónan Jafnarsel": (64.09889301241216, -21.826971272553056),
    "Krónan Vallarkór": (64.0854487078441, -21.822379330997656),
    "Krónan Lindir": (64.10251080254932, -21.873620252057236),
    "Krónan Grandi": (64.15864328795905, -21.949117186999928),
    "Krónan Hallveigarstígur": (64.1475671399666, -21.936757566966456),
    "Krónan Borgartún": (64.14831566757304, -21.898992064855193),
    "Krónan Austurver": (64.13139401690862, -21.88937902795415),
    "Krónan Hamraborg": (64.11491170786942, -21.905171874291582),
    "Krónan Garðabær": (64.09941317124606, -21.909806731692516),
    "Krónan Flatahraun": (64.07763129311066, -21.942765716339103),
    "Krónan Hvaleyrarbraut": (64.06344469095414, -21.965596678979097),
    "Krónan Norðurhella": (64.04673427118828, -21.983942988771656),
    "Krónan Skeifan": (64.13047013989885, -21.872698450597895)
}

locations_demand = {
    "DC": 0,
    "Krónan Grafarholt": 57,
    "Krónan Mosfellsbær": 60,
    "Krónan Bíldshöfi": 80,
    "Krónan Árbær": 40,
    "Krónan Jafnarsel":  62,
    "Krónan Vallarkór": 62,
    "Krónan Lindir":  78,
    "Krónan Grandi": 80,
    "Krónan Hallveigarstígur": 43,
    "Krónan Borgartún": 55,
    "Krónan Austurver": 35,
    "Krónan Hamraborg":  35,
    "Krónan Garðabær":  10,
    "Krónan Flatahraun":  60,
    "Krónan Hvaleyrarbraut":  62,
    "Krónan Norðurhella":  61,
    "Krónan Skeifan": 72
}

In [1079]:
# Haversine distance, calculate distance between two coordinates
def haversine(loc1, loc2):
    R = 6371.0  # Earth radius in kilometers
    c1 = locations_coords[loc1]
    c2 = locations_coords[loc2]
    lat1, lon1 = math.radians(c1[0]), math.radians(c1[1])
    lat2, lon2 = math.radians(c2[0]), math.radians(c2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    # road heuristic mutliplier r
    r = 1.2
    return R * c * r

cost_per_km = 1  # Cost per kilometer

dist = {(c1, c2): round(haversine(c1, c2)*cost_per_km, 2) for c1, c2 in combinations(locations, 2)}

print(dist)
print(len(locations))


{('DC', 'Krónan Grafarholt'): 2.78, ('DC', 'Krónan Mosfellsbær'): 8.61, ('DC', 'Krónan Bíldshöfi'): 0.78, ('DC', 'Krónan Árbær'): 1.14, ('DC', 'Krónan Jafnarsel'): 3.37, ('DC', 'Krónan Vallarkór'): 5.02, ('DC', 'Krónan Lindir'): 4.77, ('DC', 'Krónan Grandi'): 9.65, ('DC', 'Krónan Hallveigarstígur'): 8.34, ('DC', 'Krónan Borgartún'): 6.45, ('DC', 'Krónan Austurver'): 5.03, ('DC', 'Krónan Hamraborg'): 5.89, ('DC', 'Krónan Garðabær'): 6.8, ('DC', 'Krónan Flatahraun'): 9.98, ('DC', 'Krónan Hvaleyrarbraut'): 12.2, ('DC', 'Krónan Norðurhella'): 14.49, ('DC', 'Krónan Skeifan'): 4.06, ('Krónan Grafarholt', 'Krónan Mosfellsbær'): 6.08, ('Krónan Grafarholt', 'Krónan Bíldshöfi'): 3.13, ('Krónan Grafarholt', 'Krónan Árbær'): 2.42, ('Krónan Grafarholt', 'Krónan Jafnarsel'): 5.69, ('Krónan Grafarholt', 'Krónan Vallarkór'): 6.99, ('Krónan Grafarholt', 'Krónan Lindir'): 7.52, ('Krónan Grafarholt', 'Krónan Grandi'): 11.53, ('Krónan Grafarholt', 'Krónan Hallveigarstígur'): 10.44, ('Krónan Grafarholt', '

In [1080]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# V = vertex set
# A = arc set
# vertices 1 to n are customers
# 0 is teh depot (DC = distribution center)
# c_ij  costs of arc
# d(S) total demand in set S
# r(S) the minimum number of vehicles needed to serve all customers in S

available_vehicles = 2 # K identical vehicles
vehicle_capacity = 500 # with this capacity = C


total_demand = sum(locations_demand.values())
print("Total Demand:", total_demand)

print("Minimum number of vehicles needed", math.ceil(total_demand/vehicle_capacity))


# main travelling salesman constraints
# Variables: is location 'i' adjacent to location 'j' on the tour?

# add the location pairs as binary variables
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

##################################### gamla ###########################################
############################# CONSTRAINTS 1 AND 2 #####################################
# Constraints: two edges incident to each location

print(locations)
m.addConstrs((vars.sum('*', c) == 2 for c in locations if c != "DC"), "two_edges")

############################# CONSTRAINTS 3 AND 4 #####################################
# number of vehicles leaving depot is the same the number entering the depot
#cons34 = m.addConstrs(vars.sum("*", "DC") == available_vehicles for c in locations)

m.addConstrs((vars.sum("DC", "*") == available_vehicles * 2 for c in locations if c != "DC"), "DC_cars")

#m.addConstr(vars.sum("DC", "*") == available_vehicles, "VehiclesLeavingDC")
#m.addConstr(vars.sum("*", "DC") == available_vehicles, "VehiclesReturningDC")

################################ CONSTRAINT 5 #########################################
# demand constraint, the sum of the demands on a route should not exceed the vehicle capacity

################################### NEW ################################################


# Callback - use lazy constraints to eliminate sub-tours
# for normal TSP
def subtourelim_TSP(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(locations):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)


def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        print(tour)
        if len(tour) < len(locations):
            # add subtour elimination constr. for every pair of cities in subtour
            path_demand = sum(locations_demand[location] for location in tour)
            #print(f"path demand = {path_demand}")
            # setja r(S) = d(S)/C
            r_s = math.ceil(path_demand/vehicle_capacity)
            #print(f"r_s = {r_s}")
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-r_s)

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = locations[:]
    cycle = locations[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle



def subtour_based_on_demand(edges):
    unvisited = locations[:]
    cycle = []  # Start with an empty cycle
    max_demand = 0  # To keep track of the maximum demand of a subtour
    
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        # Calculate the demand of this cycle
        thiscycle_demand = sum(locations_demand[location] for location in thiscycle)
        # If this cycle's demand is higher than the previously stored one's, update
        if thiscycle_demand > max_demand:
            cycle = thiscycle
            max_demand = thiscycle_demand
    return cycle




# solve model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)


Total Demand: 952
Minimum number of vehicles needed 2
['DC', 'Krónan Grafarholt', 'Krónan Mosfellsbær', 'Krónan Bíldshöfi', 'Krónan Árbær', 'Krónan Jafnarsel', 'Krónan Vallarkór', 'Krónan Lindir', 'Krónan Grandi', 'Krónan Hallveigarstígur', 'Krónan Borgartún', 'Krónan Austurver', 'Krónan Hamraborg', 'Krónan Garðabær', 'Krónan Flatahraun', 'Krónan Hvaleyrarbraut', 'Krónan Norðurhella', 'Krónan Skeifan']
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 34 rows, 153 columns and 578 nonzeros
Model fingerprint: 0x55b735bc
Variable types: 0 continuous, 153 integer (153 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 4e+00]
['Krónan Norðurhella']
Presolve 

# Analysis

In [1081]:
def subtour(edges):
    unvisited = locations[:]
    cycle = locations[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
            #print(neighbors)
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

def subtours(edges):
    unvisited = locations[:]
    cycle = locations[:] # Dummy - guaranteed to be replaced
    all_cycles = []
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        print(cycle)
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
            #print(neighbors)
        if len(thiscycle) <= len(cycle):
            all_cycles.append(cycle)
            cycle = thiscycle # New shortest subtour
    return cycle

# Retrieve solution
vals = m.getAttr('x', vars)

selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
print(selected)
all_tours = subtours(selected)
#print(all_tours)

#print(tuple(selected))
#tour = subtour(selected)
#print(tour)
#for loc in tour:
#    print(loc)
#print("DC")
#assert len(tour) == len(locations)



<gurobi.tuplelist (38 tuples, 2 values each):
 ( DC                      , Krónan Grafarholt       )
 ( DC                      , Krónan Mosfellsbær      )
 ( DC                      , Krónan Bíldshöfi        )
 ( DC                      , Krónan Árbær            )
 ( Krónan Grafarholt       , Krónan Mosfellsbær      )
 ( Krónan Bíldshöfi        , Krónan Árbær            )
 ( Krónan Jafnarsel        , Krónan Vallarkór        )
 ( Krónan Jafnarsel        , Krónan Lindir           )
 ( Krónan Vallarkór        , Krónan Lindir           )
 ( Krónan Grandi           , Krónan Hallveigarstígur )
 ( Krónan Grandi           , Krónan Borgartún        )
 ( Krónan Hallveigarstígur , Krónan Borgartún        )
 ( Krónan Austurver        , Krónan Hamraborg        )
 ( Krónan Austurver        , Krónan Skeifan          )
 ( Krónan Hamraborg        , Krónan Skeifan          )
 ( Krónan Garðabær         , Krónan Flatahraun       )
 ( Krónan Garðabær         , Krónan Norðurhella      )
 ( Krónan Flatahrau

In [1082]:
# Map the solution
import folium

map = folium.Map(location=[64.14,-21.9], zoom_start = 11)

points = []
for location in tour:
  points.append(locations_coords[location])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map